In [1]:
#importing libraries
import pandas as pd
import sqlite3
import matplotlib
import seaborn

In [2]:
#configuring mysql into jupyter
%load_ext sql
%config SqlMagic.autopandas=True

In [3]:
#converting the statement PDF file to a data frame
import tabula

#path to the PDF file
pdf_path = "/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.pdf"

# password to unlock the PDF file
password = "900365"

# options for the PDF extraction
options = {
    "pages": "all",        # Extracting data from all the pages
    "password": password   # password to unlock the PDF
}

# read tables from a PDF file and save them as CSV files
#tabula.convert_into('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.pdf', 'mpesa_statement.csv', output_format='csv', **options)

df = pd.read_csv('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.csv', delimiter=';')

#dropping the unnamed column
#df = df.drop('Unnamed', axis=1)

# Print the extracted data
print(df)


                                                      ,
0     Receipt No.,Completion Time,Details,Transactio...
1     REF8MUC88I,2023-05-15 20:19:22,"Customer Trans...
2     to - 07******212 IAN MAINA",Completed,,-50.00,...
3     REF8MUC88I,2023-05-15 20:19:22,OverDraft of Cr...
4     REF9MU2IWF,2023-05-15 20:17:21,"Merchant Payme...
...                                                 ...
7416  07******245 daniel njeri",Completed,"1,050.00"...
7417  Statement Verification Code,"To verify the val...
7418                       prompts to enter the code.",
7419                                                 ,,
7420                                         KZ3FXNRZ,,

[7421 rows x 1 columns]


In [5]:
#creating a local database to store the data
conn = sqlite3.connect('mpesa_statement_database')
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS mpesa (
    `Receipt No.` VARCHAR,
    `Completion Time` DATETIME,
    Details VARCHAR,
    `Transaction Status` VARCHAR,
    `Paid In` INT,
    Withdrawn INT,
    Balance INT
)
''')
conn.commit()

In [6]:
df.to_sql('mpesa', conn, if_exists='replace', index = False)

7421

In [8]:
import os
from sqlalchemy import create_engine
# Construct the connection URL
db_url = f"sqlite:///{os.path.abspath('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database')}"

# Set the DATABASE_URL environment variable
os.environ['DATABASE_URL'] = db_url

# Create a database engine
engine = create_engine(os.environ['DATABASE_URL'])

In [9]:
%%sql
SELECT * FROM mpesa;

Done.


,","
0,"Receipt No.,Completion Time,Details,Transactio..."
1,"REF8MUC88I,2023-05-15 20:19:22,""Customer Trans..."
2,"to - 07******212 IAN MAINA"",Completed,,-50.00,..."
3,"REF8MUC88I,2023-05-15 20:19:22,OverDraft of Cr..."
4,"REF9MU2IWF,2023-05-15 20:17:21,""Merchant Payme..."
...,...
7416,"07******245 daniel njeri"",Completed,""1,050.00""..."
7417,"Statement Verification Code,""To verify the val..."
7418,"prompts to enter the code."","
7419,",,"
